In [1]:
import numpy
import sys
import nltk
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
from keras.models import Sequential
from keras.layers import Dense,Dropout,LSTM
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Using TensorFlow backend.


In [2]:
file=open('frankenstein2.txt').read()

In [3]:
def tokenize_words(input):
    input=input.lower()
    tokenizer=RegexpTokenizer(r'\w+')
    tokens=tokenizer.tokenize(input)
    filtered=filter(lambda token:token not in stopwords.words('english'),tokens)
    return ''.join(filtered)

processed_inputs=tokenize_words(file)

In [4]:
chars=sorted(list(set(processed_inputs)))
char_to_num=dict((c,i) for i,c in enumerate(chars))

In [5]:
input_len=len(processed_inputs)
vocab_len=len(chars)
print('Total number of characters',input_len)
print('total',vocab_len)

Total number of characters 4580
total 25


In [6]:
seq_length=100
x_data=[]
y_data=[]

In [7]:
for i in range(0,input_len-seq_length,1):
    in_seq=processed_inputs[i:i+seq_length]
    out_seq=processed_inputs[i+seq_length]
    x_data.append([char_to_num[char] for char in in_seq])
    y_data.append(char_to_num[out_seq])
    
n_patterns=len(x_data)
print ('total patterns:',n_patterns)


total patterns: 4480


In [8]:
X=numpy.reshape(x_data,(n_patterns,seq_length,1))
X=X/float(vocab_len)

In [9]:
y=np_utils.to_categorical(y_data)

In [10]:
model= Sequential()
model.add(LSTM(256,input_shape=(X.shape[1],X.shape[2]),return_sequences=True))
model.add(Dropout(0,2))
model.add(LSTM(256,return_sequences=True))
model.add(Dropout(0,2))
model.add(LSTM(128))
model.add(Dropout(0,2))
model.add(Dense(y.shape[1],activation='softmax'))

W0530 00:51:26.297198 139666037532480 deprecation_wrapper.py:119] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0530 00:51:26.314050 139666037532480 deprecation_wrapper.py:119] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0530 00:51:26.316809 139666037532480 deprecation_wrapper.py:119] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



In [11]:
model.compile(loss='categorical_crossentropy',optimizer='adam')

W0530 00:51:27.408966 139666037532480 deprecation_wrapper.py:119] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0530 00:51:27.434555 139666037532480 deprecation_wrapper.py:119] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



In [12]:
filepath='model_weights_saved.hdf5'
checkpoint=ModelCheckpoint(filepath,monitor='loss',verbose=1,save_best_only=True,mode='min')
desired_callbacks=[checkpoint]

In [13]:
model.fit(X,y, epochs=10,batch_size=600,callbacks=desired_callbacks)

W0530 00:51:27.590537 139666037532480 deprecation.py:323] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/math_grad.py:1250: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0530 00:51:29.697499 139666037532480 deprecation_wrapper.py:119] From /mnt/disks/user/anaconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Epoch 1/10
4480/4480 [==============================] - 11s 3ms/step - loss: 3.0790

Epoch 00001: loss improved from inf to 3.07899, saving model to model_weights_saved.hdf5
Epoch 2/10
4480/4480 [==============================] - 6s 1ms/step - loss: 2.9022

Epoch 00002: loss improved from 3.07899 to 2.90218, saving model to model_weights_saved.hdf5
Epoch 3/10
4480/4480 [==============================] - 6s 1ms/step - loss: 2.8879

Epoch 00003: loss improved from 2.90218 to 2.88793, saving model to model_weights_saved.hdf5
Epoch 4/10
4480/4480 [==============================] - 6s 1ms/step - loss: 2.8864

Epoch 00004: loss improved from 2.88793 to 2.88643, saving model to model_weights_saved.hdf5
Epoch 5/10
4480/4480 [==============================] - 6s 1ms/step - loss: 2.8851

Epoch 00005: loss improved from 2.88643 to 2.88514, saving model to model_weights_saved.hdf5
Epoch 6/10
4480/4480 [==============================] - 6s 1ms/step - loss: 2.8842

Epoch 00006: loss improved from 2.

In [14]:
filename='model_weights_saved.hdf5'
model.load_weights(filename)
model.compile(loss='categorical_crossentropy',optimizer='adam')

In [15]:
num_to_char=dict((i,c) for i,c in enumerate(chars))

In [16]:
start=numpy.random.randint(0,len(x_data)-1 )
pattern=x_data[start]
print('Random Seed:')
print("\"", ''.join([num_to_char[value] for value in pattern]), "\"")

Random Seed:
" ngsvaleattractednoticesingularlydisconsolatenumberhalfclothedchildrengatheredspokepenuryworstshapeon "


In [17]:
for i in range(1000):
    x=numpy.reshape(pattern,(1,len(pattern), 1))
    x=x/float(vocab_len)
    prediction=model.predict(x,verbose=0)
    index=numpy.argmax(prediction)
    result=num_to_char[index]
    seq_in=[num_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern=pattern[1:len(pattern)]
    


eeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee